In [38]:
class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def plus(self, v):
        new_coordinates = [x+y for x,y in zip(self.coordinates, v.coordinates)]
#         new_coordinates = []
#         n = len(self.coordinates)
#         for i in range(n):
#             new_coordinates.append(self.coordinates[i] + v.coordinates[i])
        return Vector(new_coordinates)
    
    def minus(self, v):
        new_coordinates = [x-y for x,y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)
    
    def times_scalar(self, c):
        new_coordinates = [c*x for x in self.coordinates]
        return Vector(new_coordinates)
    
    def decimal(self, d):
        # limit to 3 decimal places
        # https://stackoverflow.com/a/3928647
        new_coordinates = [round(x, d) if isinstance(x, float) else x for x in self.coordinates]
        return Vector(new_coordinates)

    def magnitude(self):
        v_square = 0
        for x in self.coordinates:
            v_square += pow(x, 2)
        return math.sqrt(v_square)
        # coordinates_squared = [x**2 for x in self.coordinates]
        # return sqrt(sum(coordinates_squared))
    
    def normalize(self):
        try:
            sc = 1./self.magnitude()
            return self.times_scalar(sc)
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')
            
    def dot_product(self, v):
        new_coordinates = [x*y for x,y in zip(self.coordinates, v.coordinates)]
        return sum(new_coordinates)
    
    def angle(self, v):
        return math.acos(self.dot_product(v) / (self.magnitude() * v.magnitude()))
    
#     def is_parallel(self, v):
#         div = self.coordinates[0] / v.coordinates[0]
#         for i in range(1, len(self.coordinates)):
#             if (div != (self.coordinates[i] / v.coordinates[i])):
#                 return False
#         return True

    def is_parallel(self, v):
        return (self.is_zero() or v.is_zero() or
               self.angle(v) == 0 or self.angle(v) == math.pi)
    
    def is_zero(self, tolerance=1e-10):
        # check if vector magnitude is zero
        return self.magnitude() < tolerance
    
    def is_orthogonal(self, v, tolerance=1e-10):
        # return 0 == self.dot_product(v) (yield False negative)
        return abs(self.dot_product(v)) < tolerance

In [39]:
import math
import numpy as np

In [3]:
my_vector = Vector([1,2,3])
my_vector2 = Vector([2,3,4])
print(my_vector)
print(my_vector.plus(my_vector2))

Vector: (1, 2, 3)
Vector: (3, 5, 7)


## Operating in Vectors
[Link](https://learn.udacity.com/courses/ud953/lessons/dc261432-62e3-4480-8f70-5f7295bf11c5/concepts/5a237c19-bd38-40bc-957a-4e0dc2a2130c)  
Addition
$\begin{bmatrix} a \\ b \end{bmatrix} + \begin{bmatrix} c \\ d \end{bmatrix} = \begin{bmatrix} a+c \\ b+d \end{bmatrix}$

Substraction
$\begin{bmatrix} a \\ b \end{bmatrix} - \begin{bmatrix} c \\ d \end{bmatrix} = \begin{bmatrix} a-c \\ b-d \end{bmatrix}$

Scalar Multiplication
$ x \begin{bmatrix} a \\ b \end{bmatrix} = \begin{bmatrix} x.a \\ x.b \end{bmatrix}$

In [4]:
vector_plus1 = Vector([8.218, -9.341])
vector_plus2 = Vector([-1.129, 2.111])
print(vector_plus1.plus(vector_plus2).decimal(3))

vector_minus1 = Vector([7.119, 8.215])
vector_minus2 = Vector([-8.223, 0.878])
print(vector_minus1.minus(vector_minus2).decimal(3))

scalar = 7.41
vector_times = Vector([1.671, -1.012, -0.318])
print(vector_times.times_scalar(scalar).decimal(3))

Vector: (7.089, -7.23)
Vector: (15.342, 7.337)
Vector: (12.382, -7.499, -2.356)


## Magnitude and Direction
[Link](https://learn.udacity.com/courses/ud953/lessons/dc261432-62e3-4480-8f70-5f7295bf11c5/concepts/5d7fc081-3aaf-4fef-8a1c-95a59cb06df9)  
Magnitude $n$ dimension  
$ \vec{v} = \begin{bmatrix} v_1 \\ v_2 \\ \vdots \\ v_n \end{bmatrix} $

$ \| \vec{v} \| = \sqrt{v_1^2 + v_2^2 \cdots + v_n^2} $  

Normalization  
$\frac{1}{\|\vec{v}\|} \vec{v} = $ unit vector in direction of $\vec{v}$

In [5]:
v_mag1 = Vector([-0.221, 7.437])
v_mag2 = Vector([8.813, -1.331, -6.247])
print(round(v_mag1.magnitude(), 3))
print(round(v_mag2.magnitude(), 3))

v_norm1 = Vector([5.581, -2.136])
v_norm2 = Vector([1.996, 3.108, -4.554])
print(v_norm1.normalize().decimal(3))
print(v_norm2.normalize().decimal(3))

7.44
10.884
Vector: (0.934, -0.357)
Vector: (0.34, 0.53, -0.777)


## Inner products
[Link](https://learn.udacity.com/courses/ud953/lessons/dc261432-62e3-4480-8f70-5f7295bf11c5/concepts/c2ce3350-ef93-4b39-9e02-c6458bc8157b)   

Dot products  
$ \vec{v} \cdot \vec{w} = \|\vec{v}\| . \|\vec{w}\| . cos \theta$  

$ \vec{v} \cdot \vec{w} = v_1w_1 + v_2w_2 + \cdots + v_nw_n $  

Angle  
$ \theta = arccos\left( \frac{\vec{v}\cdot\vec{w}}{\|\vec{v}\| . \|\vec{w}\|} \right) = arccos\left( \frac{1}{\|\vec{v}\|} \vec{v} . \frac{1}{\|\vec{w}\|} \vec{w} \right)$

In [21]:
v_dot1 = Vector([7.887, 4.138])
v_dot2 = Vector([-8.802, 6.776])
print(round(v_dot1.dot_product(v_dot2), 3))

v_dot3 = Vector([-5.955, -4.904, -1.874])
v_dot4 = Vector([-4.496, -8.755, 7.103])
print(round(v_dot3.dot_product(v_dot4), 3))

v_angle1 = Vector([3.183, -7.627])
v_angle2 = Vector([-2.668, 5.319])
print(round(v_angle1.angle(v_angle2), 3))

v_angle3 = Vector([7.35, 0.221, 5.188])
v_angle4 = Vector([2.751, 8.259, 3.985])
v_degree = math.degrees((v_angle3.angle(v_angle4)))
print(round(v_degree, 3))

-41.382
56.397
3.072
60.276


## Parallel and Orthogonal Vectors
[Link](https://learn.udacity.com/courses/ud953/lessons/dc261432-62e3-4480-8f70-5f7295bf11c5/concepts/be817cae-98ec-4125-ac9c-54de851e23fa)  

Parallel  
$\vec{v}$ and $\vec{w}$ are **parallel** if one is a scalar multiple of the other  
- $\vec{v}$ is parallel to $2\vec{v}, \frac{1}{2}\vec{v}, -\vec{v}, \vec{O}$

Orthogonal  
$\vec{v}$ and $\vec{w}$ are **orthogonal** if $\vec{v} \cdot \vec{w} = 0$  

$\vec{O}$ is **parallel** and **orthogonal** to all vectors, and the only vector **orthogonal** to itself

In [40]:
pv1 = Vector([-7.579, -7.88])
pw1 = Vector([22.737, 23.64])
print ('is parallel: ', pv1.is_parallel(pw1))
print ('is orthogonal: ', pv1.is_orthogonal(pw1))

pv2 = Vector([-2.029, 9.97, 4.172])
pw2 = Vector([-9.231, -6.639, -7.245])
print ('is parallel: ', pv2.is_parallel(pw2))
print ('is orthogonal: ', pv2.is_orthogonal(pw2))


pv3 = Vector([-2.328, -7.284, -1.214])
pw3 = Vector([-1.821, 1.072, -2.94])
print ('is parallel: ', pv3.is_parallel(pw3))
print ('is orthogonal: ', pv3.is_orthogonal(pw3))


pv4 = Vector([2.118, 4.827])
pw4 = Vector([0, 0])
print ('is parallel: ', pv4.is_parallel(pw4))
print ('is orthogonal: ', pv4.is_orthogonal(pw4))

is parallel:  True
is orthogonal:  False
is parallel:  False
is orthogonal:  False
is parallel:  False
is orthogonal:  True
is parallel:  True
is orthogonal:  True


## Projecting Vectors
[Link](https://learn.udacity.com/courses/ud953/lessons/dc261432-62e3-4480-8f70-5f7295bf11c5/concepts/f1d343b5-2795-49a5-80a8-4a18c818b77c)  
